In [250]:
metadata["soup"].head(4)

0    jealousy toy boy friendship friends Tom Hanks ...
1    board game disappearance based on children's b...
2    fishing best friend duringcreditsstinger old m...
3    based on novel interracial relationship single...
Name: soup, dtype: object

In [156]:
 
from gensim.models import Word2Vec
import os
import pandas as pd
import numpy as np
from ast import literal_eval
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import multiprocessing


In [28]:
#downloading nltk stopwords.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [11]:
 os.chdir(r"C:\Users\HP\data")

In [12]:
metadata=pandas.read_csv('movies_metadata.csv',low_memory=False)
keywords=pandas.read_csv('keywords.csv')
credits=pandas.read_csv('credits.csv')
metadata=metadata.drop([19730,29503,35587])

In [13]:
keywords['id']=keywords['id'].astype('int')
metadata['id']=metadata['id'].astype('int')
credits['id']=credits['id'].astype('int')


In [14]:
metadata=metadata.merge(keywords,on='id')
metadata=metadata.merge(credits,on='id')


In [75]:
def get_director(data):
    
    for i in data:
        if (i['job']=='Director'):
            return i["name"]
        
        return np.NaN
    
    
def get_director(data):
    
    for i in data:
        if (i['job']=='Director'):
            return i["name"]
        
        return np.NaN
    
def stemmer(data):
    #assumption takes in a tokenized form of data
    stemmer=SnowballStemmer("english")
    text=[]
    for i in data:
        
        text.append([stemmer.stem(word) for word in i])
    return text
    
    
def get_list(data):
    
    names=[i['name'] for i in data]
     
    
    if(len(names)>5):
        names=names[:5]
    
    return names



def clean_data(x):
    
    if isinstance(x,list):
        
        names=[str.lower(i.replace(" ","")) for i in data]
    if isinstance(x,str):
        x=str.lower(x.replace(" ",""))
    
    
def create_soup(data):
    return " ".join(data["keywords"])+" "+" ".join(data["cast"])+" "+data["director"]+" "+" ".join(data["genres"])

def create_soup2(data):
    return data["soup"]+" "+data["title"]

def create_soup3(data):
    
    data["tagline"]=data["tagline"].apply(remove_white)
    return data["soup"]+" "+data["tagline"]

def clean_data(x):
    names=[str(i.replace(" ","")) for i in data]
    return names;

def remove_white(data):
    if isinstance(data,str):
        data=str.lower(data.replace(" ",""))
        return data
    
    return np.NaN

def remove_stop_words(data):
    #creating a list of stop-words
    stoplist=set(stopwords.words('english'))
    text=[]
    #lower case each document ,split it by white space and filter out stopwords.
    
    for i in data:
        
        text.append([word.lower() for word in i if word not in stoplist ])
    return text

def tokenize(data):
    temp=list(data)
    temp=[str.lower(i) for i in temp]
    temp=[list(i.split(' ')) for i in temp]
    
    return temp


def model(data):
    
    model_=Word2Vec(data,window=3,min_count=1,sg=1,workers=multiprocessing.cpu_count())
    
    return model_

def model2(data):
    
    model_=Word2Vec(data,window=3,min_count=1,sg=1,workers=multiprocessing.cpu_count(),compute_loss=True,negative=15)
    
    return model_


def cleaner(data):
    data=[str.lower(i) for i in data.splitlines()]
    
    
    
def find10_most_similar(model,text):
    text=text.lower()
    list_of_similar=model.wv.most_similar(text)
    return list_of_similar

 
    

 

In [16]:
#removing nulls from the data
metadata.fillna(" ",inplace=True)

In [17]:
features=['cast','keywords','crew','genres']

for i in features:
    metadata[i]=metadata[i].apply(literal_eval)

In [19]:
metadata['director']=metadata['crew'].apply(get_director)
metadata['director'].fillna(" ",inplace=True)
metadata['director']=metadata['director'].apply(remove_white)

In [20]:
for i in features:
    metadata[i]=metadata[i].apply(get_list)

In [21]:
metadata['title']=metadata['title'].apply(remove_white )

In [22]:
metadata["soup"]=metadata.apply(create_soup,axis=1)

In [23]:
metadata["soup"]=metadata.apply(create_soup2,axis=1)

In [226]:
tokenized_data=tokenize(metadata["soup"])

In [93]:
tokenized_data=remove_stop_words(tokenized_data)

In [94]:
skipgram=model(tokenized_data)

In [179]:
 #finding similarity between all our titles and the query word
def find_similar(query):
    similar=[]
    
    #find the similarity between the query word and all of our titles in the collection.
    for i in metadata['title']:
        try:
            similarity=skipgram.wv.similarity(query,i)
            
        except Exception as e:
            pass
 
        #print("similarity between {} and {} is {}".format(query,i,similarity))
        if i == query:
            continue
            
        similar.append((i,similarity))
        
            
        #we should get all scores and find the score that lies in the 90th percentile of all the scores
    similar=list(set(similar))
    similar=pd.DataFrame(similar,columns=['title','score'])
    
    #required to sort the dataframe in the order of priority.
    
            
    return similar.sort_values(by='score',ascending=False)

    

In [243]:
#the following class will model the user session object
class user(object):
    def __init__(self):
          self.latest_search='jumanji'
            
    def similar(self):
        return find_similar(self.latest_search)
        

In [244]:
#creating the user object:
user_1=user()p

In [246]:
user_1.similar()[:100]

,title,score
444,thelife&adventuresofsantaclaus,0.942630
21522,bridgetoterabithia,0.936231
25358,pokémon:spelloftheunknown,0.932714
37832,journeytothecenteroftheearth,0.930619
12781,alittleprincess,0.929734
...,...,...
24124,rudolphthered-nosedreindeer,0.898291
30800,mostlyghostly,0.898188
11428,saintseiyaheavenchapter:overture,0.898070
391,garfield:atailoftwokitties,0.897511


In [239]:
from keras.preprocessing import text
from keras.layers import Concatenate
from keras.layers.core import Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Sequential
# visualize model structure
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

#module to generate skipgrams
from keras.preprocessing.sequence import skipgrams


In [195]:
tokenizer=text.Tokenizer()

In [199]:
#building the vocabulary for our textual data
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(metadata['soup'])

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

vocab_size = len(word2id) + 1 
embed_size = 100

wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in metadata['soup']]
print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 114364
Vocabulary Sample: [('drama', 1), ('comedy', 2), ('thriller', 3), ('romance', 4), ('action', 5), ('horror', 6), ('crime', 7), ('documentary', 8), ('adventure', 9), ('family', 10)]


In [213]:

# generate skip-grams
skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in wids]

# view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id2word[pairs[i][0]], pairs[i][0], 
          id2word[pairs[i][1]], pairs[i][1], 
          labels[i]))

(friends (576), there'snoplacelikeutopia (40164)) -> 0
(allen (126), don (151)) -> 1
(boy (541), jim (99)) -> 1
(boy (541), hasaj (67501)) -> 0
(animation (21), allen (126)) -> 1
(jim (99), biplane (29881)) -> 0
(varney (5531), sale (13861)) -> 0
(animation (21), mirchoff (14607)) -> 0
(jim (99), aportraitofcomposermortenlauridsen (70323)) -> 0
(wallace (640), cagedheat3000 (114350)) -> 0


In [214]:
word_model=Sequential()

In [215]:
word_model.add(Embedding(vocab_size, embed_size,
                         embeddings_initializer="glorot_uniform",
                         input_length=1))

word_model.add(Reshape((embed_size, )))

context_model = Sequential()
context_model.add(Embedding(vocab_size, embed_size,
                  embeddings_initializer="glorot_uniform",
                  input_length=1))
context_model.add(Reshape((embed_size,)))

model = Sequential()
model.add(Concatenate()([word_model, context_model]))
model.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))
model.compile(loss="mean_squared_error", optimizer="rmsprop")

# view model summary
print(model.summary())


SVG(model_to_dot(model, show_shapes=True, show_layer_names=False, 
                 rankdir='TB').create(prog='dot', format='svg'))

ValueError: Layer concatenate_2 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.engine.sequential.Sequential'>. Full input: [<keras.engine.sequential.Sequential object at 0x0000020308FFFE08>, <keras.engine.sequential.Sequential object at 0x00000203E655ED08>]. All inputs to the layer should be tensors.

In [ ]:
import pandas as pd
#enter filepath to the model
path=''
data=pd.read_csv(path)

In [234]:
model=Sequential()